In [1]:
import os
import json
import shutil
from pathlib import Path

# -----------------------------
# CONFIGURACIÓN
# -----------------------------
# Ruta donde están los datos ORIGINALES (raw)
ORIGIN_DIR = "/media/guest/PORT-DISK/Practicas/MicroBleeds/VALDO/Task2"

# Ruta base de nnUNet_raw (donde se creará la nueva carpeta)
NNUNET_RAW_BASE = "/media/guest/PORT-DISK/Practicas/Microbleeds/nnUNet_raw_data"

# ID y Nombre del Dataset (Formato estricto nnUNetv2: DatasetXXX_Nombre)
DATASET_ID = 888  # Puedes cambiar este número (ej. 501, 002), pero usa 3 dígitos
DATASET_NAME = "VALDO_Task2"
FOLDER_NAME = f"Dataset{DATASET_ID}_{DATASET_NAME}"

# Rutas de destino finales
TARGET_DIR = os.path.join(NNUNET_RAW_BASE, FOLDER_NAME)
IMAGES_TR_DIR = os.path.join(TARGET_DIR, "imagesTr")
LABELS_TR_DIR = os.path.join(TARGET_DIR, "labelsTr")

# -----------------------------
# 1. PREPARACIÓN DE CARPETAS
# -----------------------------
if os.path.exists(TARGET_DIR):
    print(f"[Aviso] La carpeta destino ya existe: {TARGET_DIR}")
    print("Por seguridad, borra la carpeta destino manualmente si quieres regenerarla.")
    # exit() # Descomenta esto si quieres evitar sobrescribir

os.makedirs(IMAGES_TR_DIR, exist_ok=True)
os.makedirs(LABELS_TR_DIR, exist_ok=True)

print(f"Generando dataset en: {TARGET_DIR}")

# -----------------------------
# 2. COPIA DE ARCHIVOS
# -----------------------------
# Buscamos las carpetas de sujetos (sub-XXX)
subject_folders = sorted([f for f in os.listdir(ORIGIN_DIR)
                          if f.startswith("sub") and os.path.isdir(os.path.join(ORIGIN_DIR, f))])

print(f"Se encontraron {len(subject_folders)} sujetos. Iniciando copia...")

for sub_folder in subject_folders:
    src_path = os.path.join(ORIGIN_DIR, sub_folder)
    
    # Definir rutas origen
    # NOTA: Asegúrate de que tus archivos origen coincidan con estos nombres
    src_t2s = os.path.join(src_path, f"{sub_folder}_space-T2S_desc-masked_T2S.nii.gz")
    src_t1  = os.path.join(src_path, f"{sub_folder}_space-T2S_desc-masked_T1.nii.gz")
    src_t2  = os.path.join(src_path, f"{sub_folder}_space-T2S_desc-masked_T2.nii.gz")
    src_lbl = os.path.join(src_path, f"{sub_folder}_space-T2S_CMB.nii.gz")

    # Comprobar que existen antes de copiar
    if not all(os.path.exists(p) for p in [src_t2s, src_t1, src_t2, src_lbl]):
        print(f"[ERROR] Faltan archivos en {sub_folder}. Saltando...")
        continue

    # Definir rutas destino (Renombrado nnUNetv2)
    # T2S -> 0000
    # T1  -> 0001
    # T2  -> 0002
    dst_t2s = os.path.join(IMAGES_TR_DIR, f"{sub_folder}_0000.nii.gz")
    dst_t1  = os.path.join(IMAGES_TR_DIR, f"{sub_folder}_0001.nii.gz")
    dst_t2  = os.path.join(IMAGES_TR_DIR, f"{sub_folder}_0002.nii.gz")
    dst_lbl = os.path.join(LABELS_TR_DIR, f"{sub_folder}.nii.gz")

    # Copiar (shutil.copy2 preserva metadatos)
    print(f"Procesando {sub_folder}...", end="\r")
    shutil.copy2(src_t2s, dst_t2s)
    shutil.copy2(src_t1, dst_t1)
    shutil.copy2(src_t2, dst_t2)
    shutil.copy2(src_lbl, dst_lbl)

print(f"\nCopia finalizada.")

# -----------------------------
# 3. GENERAR DATASET.JSON (Formato v2)
# -----------------------------
# CAMBIOS CLAVE v2:
# - "modality" -> "channel_names"
# - Las claves de channel_names deben ser strings ("0", "1")
# - "labels" debe ser mapa Nombre -> Int
# - Añadir "file_ending"

json_dict = {
    "name": DATASET_NAME,
    "description": "Microbleeds detection VALDO Task2 Multi-modal",
    "tensorImageSize": "3D",
    "reference": "VALDO Challenge",
    "licence": "",
    "release": "1.0",
    
    # AQUÍ ESTÁ EL CAMBIO CLAVE PARA v2:
    "channel_names": {
        "0": "T2S",
        "1": "T1",
        "2": "T2"
    },
    
    # Mapeo de etiquetas (Nombre -> Valor píxel)
    "labels": {
        "background": 0,
        "CMB": 1
    },
    
    "numTraining": len(subject_folders),
    "file_ending": ".nii.gz",
    
    # En v2, 'training' es opcional si la carpeta imagesTr está bien,
    # pero ponerlo ayuda a depurar.
    "training": [
        {
            "image": f"./imagesTr/{sub}.nii.gz",
            "label": f"./labelsTr/{sub}.nii.gz"
        }
        for sub in subject_folders
    ]
    # NOTA: En la sección 'training', NO se pone _0000 en el nombre del archivo.
    # nnUNet asume los sufijos de canal automáticamente.
}

json_path = os.path.join(TARGET_DIR, "dataset.json")
with open(json_path, 'w') as f:
    json.dump(json_dict, f, indent=4)

print(f"Dataset v2 creado en: {TARGET_DIR}")


Generando dataset en: /media/guest/PORT-DISK/Practicas/Microbleeds/nnUNet_raw_data/Dataset888_VALDO_Task2
Se encontraron 72 sujetos. Iniciando copia...
Procesando sub-327...
Copia finalizada.
Dataset v2 creado en: /media/guest/PORT-DISK/Practicas/Microbleeds/nnUNet_raw_data/Dataset888_VALDO_Task2
